In [1]:
import requests
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import re

# 한국어 지명 필터 목록 생성
def get_korean_place_names():
    response = requests.get('https://nominatim.openstreetmap.org/search?format=json&q=대한민국')
    places = response.json()
    place_names = set()
    for place in places:
        place_names.add(place['display_name'])
    return place_names

korean_place_names = get_korean_place_names()

# 한국어 사람 이름 필터 목록 생성
def get_korean_names():
    url = 'https://raw.githubusercontent.com/medialab/kor-names/master/korean_names.txt'
    response = requests.get(url)
    names = response.text.split('\n')
    return set(name.strip() for name in names if name.strip())

korean_names = get_korean_names()

# 한국어 시간 관련 단어 필터 목록 생성
def get_korean_time_words():
    time_words = set(['아침', '오후', '저녁', '밤', '시간', '분', '초', '요일'])
    return time_words

korean_time_words = get_korean_time_words()

# 텍스트에서 지명, 사람 이름, 시간 관련 단어를 필터링
def clean_text(text):
    # 한국어 형태소 분석기 (예: KoNLPy) 필요
    from konlpy.tag import Okt
    okt = Okt()
    
    # 단어 토큰화
    words = okt.morphs(text)
    
    # 불용어, 지명, 사람 이름, 시간 관련 단어를 제거
    filtered_words = [word for word in words if word not in korean_place_names and
                      word not in korean_names and
                      word not in korean_time_words]
    
    # 필터링된 단어를 다시 문자열로 변환
    return ' '.join(filtered_words)

# 데이터프레임 예시 생성
data = {
    '눈': 508,
    '지방': 435,
    '내일': 391,
    '기온': 365,
    '오늘': 360,
    '날씨': 329,
    '구름': 277,
    '영하': 274,
    '비': 245,
    '아침': 239,
    '낮': 204,
    '전국': 186,
    '해상': 170,
    '영상': 166,
    '중부': 145,
    '서울': 144,
    '밤': 125,
    '예상': 119,
    '추위': 118,
    '지역': 118,
    '기자': 111,
    '호남': 108,
    '겨울': 98,
    '기상': 98,
    '최고': 98,
    '주의보': 94,
    '이번': 93,
    '충청': 85,
    '물결': 84,
    '오후': 83,
    '영동': 79,
    '동해': 79,
    '주': 78,
    '캐스터': 77,
    '전반': 76,
    '모레': 74,
    '서해': 68,
    '영향': 68,
    '조석': 66,
    '내륙': 65,
    '곳': 64,
    '계속': 63,
    '안팎': 58,
    '기압': 58,
    '다음': 57,
    '주말': 57,
    '바람': 55,
    '남부': 52,
    '때': 51,
    '사이': 51,
    '우리나라': 50,
    '휴일': 50,
    '제주도': 50,
    '서해안': 49,
    '앵커': 47,
    '뉴스': 46,
    '폭풍': 43,
    '뒤': 43,
    '오전': 42,
    '일부': 41,
    '건조': 41,
    '시작': 40,
    '지금': 39,
    '상태': 39,
    '대부분': 39,
    '가운데': 38,
    '울릉도': 38,
    '산간': 38,
    '차량': 38,
    '약간': 38,
    '경기': 37,
    '이기문': 36,
    '나라': 35,
    '도로': 34,
    '부분': 34,
    '비구름': 33,
    '남해': 33,
    '먼바다': 32,
    '중심': 32,
    '영남': 32,
    '제주': 32,
    '예년': 31,
    '기상청': 31,
    '북부': 31,
    '안개': 31,
    '발효': 31,
    '바다': 30,
    '길': 29,
    '정도': 29,
    '서쪽': 29,
    '연휴': 28,
    '동안': 27,
    '곳곳': 27,
    '부근': 27,
    '한파': 27,
    '중': 26,
    '가량': 26,
    '강원도': 26,
    '전망': 25,
    '새해': 25,
    '일교차': 25,
    '강원': 24,
    '대륙': 24,
    '날': 22,
    '속': 22,
    '수요일': 22,
    '한강': 22,
    '후': 21,
    '운행': 21,
    '류근찬': 21,
    '교통': 20,
    '대설': 20,
    '금요일': 20,
    '출근길': 20,
    '이상': 19,
    '폭설': 19,
    '세력': 19,
    '서부': 19,
    '북쪽': 19,
    '동해안': 19,
    '조심': 19,
    '독도': 19,
    '온도': 19,
    '정보': 19,
    '빙판': 18,
    '새벽': 18,
    '보도': 18,
    '밖': 18,
    '홍사훈': 17,
    '체감': 17,
    '예보': 16,
    '비교': 16,
    '사고': 16,
    '월요일': 16,
    '봄': 16,
    '이남': 15,
    '목요일': 15,
    '한때': 15,
    '소식': 14,
    '앞': 14,
    '기압골': 14,
    '하루': 14,
    '사람': 14,
    '사흘': 13,
    '중국': 13,
    '발생': 13,
    '분포': 13,
    '올': 13,
    '얼음': 13,
    '홍': 12,
    '양': 12,
    '유의': 12,
    '해제': 12,
    '화요일': 12,
    '형성': 12,
    '현상': 12,
    '강추위': 12,
    '김종진': 11,
    '첫날': 11,
    '황현정': 11,
    '처음': 11,
    '경우': 11,
    '통과': 11,
    '북서쪽': 11,
    '산불': 11,
    '등압선': 11,
    '수은주': 11,
    '공기': 11,
    '주변': 11,
    '조금': 11,
    '속도': 11,
    '일요일': 11,
    '아래': 11,
    '혼잡': 10,
    '자리': 10,
    '발달': 10,
    '철원': 10,
    '이동': 10,
    '동쪽': 10,
    '경북': 10,
    '적설량': 10,
    '토요일': 9,
    '가능': 9,
    '불편': 9,
    '눈발': 9,
    '통행': 9,
    '영서': 9,
    '평년': 9,
    '규모': 9,
    '돌풍': 9,
    '화재': 9,
    '본격': 9,
    '경보': 9,
    '폭': 9,
    '대관령': 9,
    '접근': 9,
    '감기': 9,
    '모습': 9,
    '설날': 8,
    '동반': 8,
    '보관': 8,
    '시민': 8,
    '남해안': 8,
    '남서쪽': 8,
    '해당': 8,
    '포함': 8,
    '남쪽': 8,
    '차례': 8,
    '꽃': 8,
    '걱정': 7,
    '사후': 7,
    '강풍': 7,
    '관광객': 7,
    '초속': 7,
    '전': 7,
    '차': 7,
    '제외': 7,
    '남': 7,
    '대설주의보': 7,
    '서': 7,
    '손': 7,
    '요즘': 7,
    '간격': 7,
    '이틀': 7,
    '대': 7,
    '어제': 7,
    '찬바람': 7,
    '스키장': 7,
    '순서': 7,
    '춘천': 7,
    '항공': 7,
    '거리': 7,
    '도심': 7,
    '마지막': 6,
    '박은정': 6,
    '발': 6,
    '저녁': 6,
    '중서부': 6,
    '대비': 6,
    '피해': 6,
    '변화': 6,
    '초순': 6,
    '고압': 6,
    '최저': 6,
    '소': 6,
    '라니냐': 6,
    '박상범': 6,
    '기록': 6,
    '미시령': 6,
    '월동': 6,
    '퇴근길': 6,
    '눈길': 6,
   
}

df = pd.DataFrame(data)

# 데이터프레임의 'text' 열에 clean_text 함수 적용
df['cleaned_text'] = df['text'].apply(clean_text)

print(df[['cleaned_text']])



ModuleNotFoundError: No module named 'nltk'